In [1]:
#%%
try:
    import os
    import sys
    import warnings
    warnings.filterwarnings("ignore")
    print(f'Default working directory {os.getcwd()}')
    #change the directory to working directory
    #path =r"C:\Work_Anaconda\TripAdvisor\All_script_voting"
   # path =r"C:\Users\arshraj.randhawa\Desktop\work2022\reikit model building"
    os.chdir(path)
    print(f'Current working directory {os.getcwd()}')
    #check the files in directory
    # print(f'Files in the current working directory are:\n{os.listdir()}')
except Exception as e:
    print(f'Error while setting up working directory : {sys.exc_info()[0]} , {e}')

In [2]:

print('Importing Libraries & functions')
try :
    import warnings
    warnings.filterwarnings("ignore")
    import spacy
    #if spacy is not installed >>
    # pip install -U spacy
    # python -m spacy download en_core_web_lg
    import pandas as pd
    import joblib
    # pip install joblib
    import numpy as np
#     import glob
    from sklearn.metrics import accuracy_score
    from support_file2 import *
    from sklearn.preprocessing import LabelEncoder
    from nltk.tokenize import sent_tokenize
    from sklearn.model_selection import train_test_split
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn import metrics
    from sklearn.metrics import confusion_matrix
    from sklearn.feature_selection import SelectFromModel
    from sklearn.ensemble import ExtraTreesClassifier, GradientBoostingClassifier
    from sklearn.linear_model import SGDClassifier
    from sklearn.naive_bayes import ComplementNB
    from sklearn.metrics import precision_score
    from sklearn.metrics import recall_score
    from sklearn.metrics import f1_score
    print('Librabries and functions imported successfull')
except Exception as e:
    print(f'Error while importing librabries {sys.exc_info()[0]} , {e}')

Importing Libraries & functions


NameError: name 'sys' is not defined

In [3]:
print('Importing new training data from Database')
try:
    #read data from the source
    # df = pd.read_sql_query()

    df = pd.read_excel('Lysol SS 8th - 14th Sept.xlsx')
#         df = pd.read_pickle('ta_snippet_10032021.pkl')
    #only the 4 columns are necessary
    df = df[['ReviewID','Month', 'Title', 'Review', 'FullText','Rating','Title_Review','Level1','Level2','Level3']]
    head, shape, dtyp, colms, nul = dfinfo(df)
    #sort
    df.columns = [col.lower() for col in colms]
    df.sort_values(['reviewid', 'title'], inplace=True)
    df.dropna(subset=['review'],inplace=True)
    df.reset_index(drop=True, inplace=True)
    #lowering all text columns
    for col in ['title','month','review', 'fulltext', 'title_review']:
        df[col] = df[col].str.lower()
        df[col] = df[col].str.strip()
    del col
except Exception as e:
    print(f"Error while fetching the data data : {sys.exc_info()[0]} : {e}")
# df = df[['userreviewid', 'reviewtext']].drop_duplicates('userreviewid')

Importing new training data from Database


NameError: name 'sys' is not defined

In [4]:
len(df.groupby("level1").size())

8

In [5]:
df.groupby("level1").size()

level1
Adverse Event               76
Brand Perception          1252
Functionality             4980
Manufacturing & Design     704
Platform Experience        949
Pricing                    759
Product Features          1485
Usability                  239
dtype: int64

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10444 entries, 0 to 10443
Data columns (total 10 columns):
reviewid        10444 non-null object
month           10444 non-null object
title           10389 non-null object
review          10444 non-null object
fulltext        2392 non-null object
rating          10444 non-null int64
title_review    10444 non-null object
level1          10444 non-null object
level2          10444 non-null object
level3          10444 non-null object
dtypes: int64(1), object(9)
memory usage: 816.1+ KB


In [7]:

# =========================Pre processing======================================

print('Preproceing starts...')
try:
    df['review'] = df['review'].apply(cleantext)
    df = preprocess(df, 'review')
    df['cc1'] = df['cc1'].replace('', np.nan)
    head, shape, dtyp, colms, nul = dfinfo(df)
    nullreco = df[df['cc1'].isna()]
    #drop nul values
    dupli = df[df.duplicated(subset = ['reviewid', 'cc1'], keep=False)]
    df = df.drop_duplicates(subset=['reviewid', 'cc1']).reset_index(drop=True)
    df.reset_index(drop=True, inplace=True)
    #preparing dictionary d
    head, shape, dtyp, colms, nul = dfinfo(df)
    colms.remove('cc')
    d = uniq(df[colms])
    colms = list(df.columns)
    df['word'] = df['cc1'].str.split().str.len()
    df['len'] = df['cc1'].apply(len)
#     df = df[['reviewid', 'snippet', 'theme', 'sentiment', 'cc', 'cc1', '#word', 'len']]
    print('\nPreproceing of new Done')
except Exception as e:
    print(f'Error in preprocessing: {sys.exc_info()[0]} , {e}')


Preproceing starts...
removed emojis
removed punctuation
removed blank lines
lematization of text is in process...
lemmmatized text
removed stop words
removed single words
removed digits
removed white spaces
reviewid>month>title>review>fulltext>rating>title_review>level1>level2>level3>cc1>Error in preprocessing: <class 'TypeError'> , object of type 'float' has no len()


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10444 entries, 0 to 10443
Data columns (total 13 columns):
reviewid        10444 non-null object
month           10444 non-null object
title           10389 non-null object
review          10444 non-null object
fulltext        2392 non-null object
rating          10444 non-null int64
title_review    10444 non-null object
level1          10444 non-null object
level2          10444 non-null object
level3          10444 non-null object
cc              10444 non-null object
cc1             10389 non-null object
word            10389 non-null float64
dtypes: float64(1), int64(1), object(11)
memory usage: 1.0+ MB


In [9]:
df.groupby('month').size()

month
april     2234
august    2389
july      2142
june      1942
may       1737
dtype: int64

In [10]:
# =============================================================================
print('preparing data to train the model')
try:
    le = LabelEncoder()
    df['theme_enco'] = le.fit_transform(df['level1'])
    joblib.dump(le, 'le')
    #prepare X
    df['cc1']=df['cc1'].values.astype('U')
    test=df[df.month=="august"]
    train=df[df.month!="august"]
    x=df["cc1"]
    y=df["theme_enco"]
    X_train=train["cc1"]
    X_test=test["cc1"]
    y_train=train['theme_enco']
    y_test=test['theme_enco']
#     x = df['cc1']
    #prepare Y
#     y =df['theme_enco']
    # y =df1['theme']
#     X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=1, random_state=42)
    
#     y_trainindex = y_train.index
#     y_testindex = y_test.index
except Exception as e:
    print(f'Error preparing data to train the model :{sys.exc_info()[0]} , {e}')

# =============================================================================

preparing data to train the model


In [11]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(8055,)
(2389,)
(8055,)
(2389,)


In [12]:
try:
    print('Started feature engineering')
    tf = TfidfVectorizer(ngram_range=(1, 2))
    tf.fit(x)
    joblib.dump(tf, 'tf', compress=5)
    x_train = tf.transform(X_train)
    x_test = tf.transform(X_test)
    print('Feature selection going on, it may take a while')
    selector = SelectFromModel(ExtraTreesClassifier(n_estimators=100,random_state=2021)).fit(x_train, y_train)
    joblib.dump(selector, 'selector', compress=5)
    # selector = joblib.load('selector')
    x_train = selector.transform(x_train)
    x_test = selector.transform(x_test)
    print('Feature engineering is done successfully')
except Exception as e:
    print(f'Error feature engineering : {sys.exc_info()[0]} , {e}')
# =============================================================================

Started feature engineering
Feature selection going on, it may take a while
Feature engineering is done successfully


In [13]:
x_train.shape

(8055, 8502)

In [14]:
# =============================================================================
try:
    print("Training SGD model")
    m_sgd = SGDClassifier(loss='modified_huber', class_weight='balanced', random_state=2022)
    m_sgd.fit(x_train, y_train)
    joblib.dump(m_sgd, 'm_sgd', compress=5)
    
    print("Training GBoosting model")
    m_gbt = GradientBoostingClassifier(loss='deviance', random_state=2022)
    #exponential loss require 2 class
    m_gbt.fit(x_train, y_train)
    joblib.dump(m_gbt, 'm_gbt', compress=5)

    print("Training ComplementNB model")    
    m_cnb = ComplementNB()
    m_cnb.fit(x_train, y_train)
    joblib.dump(m_cnb, 'm_cnb', compress=5)

    # print("Voting algorithm running")
    # from sklearn.ensemble import VotingClassifier
    # vclassifier = VotingClassifier(estimators=[('sgd', m_sgd), ('gbt', m_gbt), ('cnb', m_cnb)], voting='soft')
    # vclassifier =  vclassifier.fit(x_train, y_train)
    # joblib.dump(vclassifier, 'vclassifier', compress=5)
    print('All ML Model trained dumped successfully')
except Exception as e:
    print(f'Error in model training model : {sys.exc_info()[0]} , {e}')


Training SGD model
Training GBoosting model
Training ComplementNB model
All ML Model trained dumped successfully


In [15]:
print('Running the prediction on data')    


Running the prediction on data


In [16]:

pred_sgd = m_sgd.predict_proba(x_test)
# pred_sgd = m_gbt.predict_proba(x_test)
# pred_sgd = m_cnb.predict_proba(x_test)
pred = pd.DataFrame(data=pred_sgd, columns=le.classes_)
pred['max'] = np.max(pred, axis=1)
pred['max'] = np.max(pred, axis=1)
pred['theme'] = pred.idxmax(axis=1)
test['m_sgd'] = pred['max'].tolist()
test['m_sgd_prob'] = pred['theme'].tolist()
theme=pd.DataFrame(le.inverse_transform(y_test),columns=["actual_theme"])
print("m_sgd model",accuracy_score(pred["theme"],theme))
print(confusion_matrix(pred["theme"],theme))
print('F1: %f' % f1)

m_sgd model 0.6848053578903307
[[  6   0   8   0   0   0   1   0]
 [  0 178  67  12   6   6  13   2]
 [  2  53 881  26  20  23  78  11]
 [  0  15  42 113  12   1   8   6]
 [  1  17  38  10 165   9   7   3]
 [  0   8  43   2   4  99   9   1]
 [  3   4 117  13  12   9 178   3]
 [  1   0  17   4   0   3   3  16]]


NameError: name 'f1' is not defined

In [ ]:


# pred_sgd = m_sgd.predict_proba(x_test)
pred_sgd = m_gbt.predict_proba(x_test)
# pred_sgd = m_cnb.predict_proba(x_test)
pred = pd.DataFrame(data=pred_sgd, columns=le.classes_)
pred['max'] = np.max(pred, axis=1)
pred['max'] = np.max(pred, axis=1)
pred['theme'] = pred.idxmax(axis=1)
test['m_gbt'] = pred['max'].tolist()
test['m_gbt_prob'] = pred['theme'].tolist()
theme=pd.DataFrame(le.inverse_transform(y_test),columns=["actual_theme"])
print("m_gbt model",accuracy_score(pred["theme"],theme))
print(confusion_matrix(pred["theme"],theme))
precision = precision_score(pred["theme"],theme,pos_label='positive', average='micro')
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(pred["theme"],theme,pos_label='positive', average='micro')
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(pred["theme"],theme,pos_label='positive', average='micro')
print('F1: %f' % f1)

In [ ]:
pred_sgd = m_cnb.predict_proba(x_test)
pred = pd.DataFrame(data=pred_sgd, columns=le.classes_)
pred['max'] = np.max(pred, axis=1)
pred['max'] = np.max(pred, axis=1)
pred['theme'] = pred.idxmax(axis=1)
test['m_cnb'] = pred['max'].tolist()
test['m_cnb_prob'] = pred['theme'].tolist()
theme=pd.DataFrame(le.inverse_transform(y_test),columns=["actual_theme"])
print("m_cnb model",accuracy_score(pred["theme"],theme))
print(confusion_matrix(pred["theme"],theme))
precision = precision_score(pred["theme"],theme,pos_label='positive', average='micro')
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(pred["theme"],theme,pos_label='positive', average='micro')
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(pred["theme"],theme,pos_label='positive', average='micro')
print('F1: %f' % f1)

In [ ]:
# test.to_csv("sampleoutout.csv")

In [1]:
 import spacy